# train_df_icdar

In [ ]:
Setto un seed -> i risultati dovrebbero essere sempre gli stessi quando seleziono train e val ->
ho verificato che se lancio due volte in momenti diversi ottengo la stessa lista di writers in validation

X
sto prendendo writer, isEng etc dal nome del file e sto associando con le labels
prendo le labels da train_answers.csv

Problema: potrebbe essere che train_answers e train abbiano gli stessi scrittori  (handcrafted features da train è associato correttamente alle labels) 
ma che train_answers e i filenames delle immagini non seguano lo stesso ordinamento degli scrittori?
Come posso verificare che sto associando le immagini corrette?
Posso addestrare un modello che usi cnn. Se efficace non ci dovrebbero essere errori
posso fare un modello che matcha immagini e features
controllo da dove sono scaricati i dati
posso riestrarre le features e compararle con quelle già estratte

problema: potrebeb essere che train answers e train abbiano scrittori ordinati diversamente? Improbabile perchè sono tutti e due dataset di kaggle e FE funziona bene (80% val accuracy)



train_answers contiene 282 scrittori.
Number of times 'male' is 0: 143
Number of times 'male' is 1: 139

X
Controllare che l'associazione con isEng sia corretta:
0 -> arabo
1 -> inglese
Posso verificarlo visivamente su alcuni esempi

In [ ]:
I checked that there are no missing values:
All writers have both same_text=1 and same_text=0.
All writers have both isEng=1 and isEng=0.

ho verificato di avere 4 esempi per ogni scrittore

I need to check that 90% of writers is with train=1 -> the fractionof row with train=1 is exactly 90%
I need to check that a writer is only in one group -> i counted the unique values for each group of rows that share a writer. I have always one unique value 

In [ ]:
I load the images from the following folders: ['1_50', '51_100', '101_150', '151_200', '201_250', '251_300']

Sto prendendo writer,isEng e same dal nome del file.

Come verificare se sto prendendo il valore giusto? Posso guardare degli esempi: 
-per writer=1 i primi due sono arabi isEng=0 e i secondi due sono inglesi isEng=1. Per writer 1 e writer 2 v=4 ha lo stesso testo mentre v=3 ha testo diverso. Per l'arabo v=2 hanno testo uguale (mi sembra) mentre v=1 ha testo diverso
-lo stesso vale per writer 253 e 254
-> sono ragionevolmente sicuro che sia tutto a posto

ho verificato che non ci siano missing values. Tutto a posto.
ho verificato di avere 4 esempi per ogni scrittore


In [ ]:
Associating the train_file_df with the train_df dataframe.
- Devo verificare che sto associando i percorsi giusti -> guardo il nome dei file e vedo se le colonne corrispondenti (writer, isEng etc ..) hanno il valore corretto. 
Ho controllato per 10 righe prese a caso e l'associazione è corretta


Associating with the sex_df (adding the sex column)
- to confirm that the merging is working i randomly sample rows from sex_df and train_df and check that the writer and sex columns are shared -> confirmed
- ripeti tutti i test sul nuovo dataset ->
Number of times 'male' is 0: 572
Number of times 'male' is 1: 556
Number of times 'male' is something else: 0
All writers have both same_text=1 and same_text=0.
All writers have both isEng=1 and isEng=0.
Number of rows where train == 1: 1012
Fraction of rows where train == 1: 0.90
The train column is constant for all writers.
Each unique writer value occurs on exactly 4 rows.

Prima di lanciare il codice ho fatto push su github
Salvo con seed e descrizione del dataset in un log file di tipo json 